In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, GRU, Bidirectional, Dense, RepeatVector, TimeDistributed


import warnings
warnings.filterwarnings('ignore')

In [17]:
df = pd.read_csv('TurinAHU.csv')
df["Timestamp"] = pd.to_datetime(df["Timestamp"])

# Extracting day, month, year, and time into separate columns
df["day"] = df["Timestamp"].dt.day
df["month"] = df["Timestamp"].dt.month
df["year"] = df["Timestamp"].dt.year
df["time"] = df["Timestamp"].dt.hour * 3600 + df["Timestamp"].dt.minute * 60 + df["Timestamp"].dt.second

# Dropping the "timestamp" column
df = df.drop("Timestamp", axis=1)

# Reordering the columns
cols = df.columns.tolist()
cols = ["time", "day", "month", "year"] + cols[:-4]
df = df[cols]
df

,time,day,month,year,T_Supply,T_Return,SP_Return,T_Saturation,T_Outdoor,RH_Supply,RH_Return,RH_Outdoor,Energy,Power
0,79200,14,10,2019,19.859999,20.469999,18.5,19.020000,20.299999,71.110001,58.919998,79.5,0.0,0.0
1,80100,14,10,2019,19.855000,20.430000,18.5,19.020000,20.299999,71.320000,59.000000,82.0,0.0,0.0
2,81000,14,10,2019,19.850000,20.410000,18.5,19.020000,20.299999,71.470001,59.109997,79.5,0.0,0.0
3,81900,14,10,2019,19.840000,20.379999,18.5,19.080000,20.299999,71.439995,59.309998,77.0,0.0,0.0
4,82800,14,10,2019,19.830000,20.350000,18.5,19.080000,20.299999,71.580002,59.559998,79.5,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33883,74700,14,4,2021,19.539999,20.004999,20.5,19.619999,14.700000,39.020000,27.930000,57.0,0.0,0.0
33884,75600,14,4,2021,19.520000,19.949999,20.5,19.539999,13.700000,39.020000,28.090000,57.0,0.0,0.0
33885,76500,14,4,2021,19.430000,19.955000,20.5,19.420000,13.700000,39.399998,27.930000,57.0,0.0,0.0
33886,77400,14,4,2021,19.420000,19.920000,20.5,19.400000,13.700000,39.599998,28.039999,57.0,0.0,0.0


In [18]:
############################################################# STATEFUL LSTM #############################################################
# Scale the data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(df)

# Prepare the data for training
X = []
y = []
n_future = 2 # number of timesteps to predict
n_past = 96 # number of timesteps to use as input
batch_size = 96
for i in range(n_past, len(data_scaled) - n_future + 1):
    X.append(data_scaled[i - n_past:i, :])
    y.append(data_scaled[i:i + n_future, :])

X = np.array(X)
y = np.array(y)

# Split the data into training and testing sets
split_index = int(len(X) * 0.7)
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

# Define the model architecture
model = Sequential()
model.add(LSTM(14, batch_input_shape=(batch_size, X_train.shape[1], X_train.shape[2]), activation='tanh', stateful=True, return_sequences=True))
model.add(LSTM(28, activation='tanh', dropout=0.5, stateful=True, return_sequences=True))
model.add(LSTM(28, activation='tanh', dropout=0.5, stateful=True))
model.add(RepeatVector(n_future))
model.add(LSTM(28, activation='tanh', dropout=0.5, stateful=True, return_sequences=True))
model.add(LSTM(28, activation='tanh', dropout=0.5, stateful=True, return_sequences=True))
model.add(TimeDistributed(Dense(X_train.shape[2], activation='linear')))
model.compile(optimizer='adam', loss='mse')

# Train the model
epochs = 500
for i in range(epochs):
    print('Epoch', i+1, '/', epochs)
    history = model.fit(X_train, y_train, epochs=100, batch_size=batch_size, validation_split=0.2, shuffle=False)
    model.reset_states()

# Evaluate the model
y_pred = model.predict(X_test, batch_size=batch_size)
y_pred_rescaled = scaler.inverse_transform(y_pred.reshape(y_pred.shape[0]*y_pred.shape[1], y_pred.shape[2]))
y_test_rescaled = scaler.inverse_transform(y_test.reshape(y_test.shape[0]*y_test.shape[1], y_test.shape[2]))

mae = np.mean(np.abs(y_pred_rescaled - y_test_rescaled), axis=0)
mape = np.mean(np.abs((y_pred_rescaled - y_test_rescaled) / y_test_rescaled), axis=0) * 100
rmse = np.sqrt(np.mean(np.square(y_pred_rescaled - y_test_rescaled), axis=0))

print('MAE:', mae)
print('MAPE:', mape)
print('RMSE:', rmse)

Shape of X_train: (23653, 96, 1, 14)


ValueError: Input 0 of layer "lstm_23" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (96, 28)